In [1]:
import sys
print("💡 Current Kernel:", sys.executable)
import sentencepiece
print("✅ sentencepiece is now active!")


💡 Current Kernel: C:\Users\udeko\anaconda3\python.exe
✅ sentencepiece is now active!


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load FLAN-T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_t5 = T5ForConditionalGeneration.fr=om_pretrained("google/flan-t5-base")

print("✅ Model and tokenizer loaded successfully!")


In [ ]:
def generate_answer(question, context):
    prompt = f"question: {question}\ncontext: {context}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model_t5.generate(**inputs, max_new_tokens=60)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Pull a question + context from your dataset
import pandas as pd

path = r"C:\Users\udeko\.cache\kagglehub\datasets\frankossai\natural-questions-dataset\versions\1"
df = pd.read_csv(f"{path}/Natural-Questions-Filtered.csv")

sample_q = df.iloc[1]["question"]
sample_context = df.iloc[1]["long_answers"]

generated_answer = generate_answer(sample_q, sample_context)

print("🧠 Question:", sample_q)
print("💬 Generated Answer:", generated_answer)
print("✅ True Answer:", df.iloc[1]['short_answers'])


In [ ]:
import numpy as np

def evaluate_model(df, n_samples=10):
    correct = 0
    total = min(n_samples, len(df))

    for i in range(total):
        q = df.iloc[i]["question"]
        context = df.iloc[i]["long_answers"]
        true_answer = str(df.iloc[i]["short_answers"]).strip().lower()
        
        generated = generate_answer(q, context).strip().lower()
        
        is_correct = true_answer in generated or generated in true_answer
        correct += int(is_correct)
        
        print(f"\n🧠 Q{i+1}: {q}")
        print(f"💬 Generated: {generated}")
        print(f"✅ True: {true_answer}")
        print(f"🎯 Match: {'Yes' if is_correct else 'No'}")

    accuracy = (correct / total) * 100
    print(f"\n📊 Accuracy over {total} samples: {accuracy:.2f}%")

# Run evaluation on first 10 samples
evaluate_model(df, n_samples=10)


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Use a lightweight embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all long answers
embeddings = embedder.encode(df['long_answers'].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings).astype('float32')

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("✅ Indexed", index.ntotal, "contexts.")


In [ ]:
import pandas as pd

# Path to your dataset
path = r"C:\Users\udeko\.cache\kagglehub\datasets\frankossai\natural-questions-dataset\versions\1"

# Load the filtered dataset
df = pd.read_csv(f"{path}/Natural-Questions-Filtered.csv")
print("✅ Dataset loaded. Rows:", len(df))


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = embedder.encode(df['long_answers'].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings).astype('float32')

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("✅ Indexed", index.ntotal, "contexts.")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load FLAN-T5 model and tokenizer again
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_t5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

print("✅ Model and tokenizer reloaded successfully!")


In [ ]:
def generate_answer(question, context):
    prompt = f"question: {question}\ncontext: {context}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model_t5.generate(**inputs, max_new_tokens=60)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
def rag_answer(question, k=a3):
    contexts = retrieve_context(question, k)
    combined_context = " ".join(contexts)
    return generate_answer(question, combined_context)

# Test example
query = "who is the president of un general assembly"
print("🧠 Question:", query)
print("💬 RAG Answer:", rag_answer(query))


In [ ]:
def rag_answer(question, k=3):
    contexts = retrieve_context(question, k)
    combined_context = " ".join(contexts)
    return generate_answer(question, combined_context)

# Test example
query = "who is the president of un general assembly"
print("🧠 Question:", query)
print("💬 RAG Answer:", rag_answer(query))


In [ ]:
print(rag_answer("who is the president of un general assembly", k=5))


In [ ]:
def rag_answer(question, k=5):
    contexts = retrieve_context(question, k)
    combined_context = " [SEP] ".join(contexts)  # separates retrieved chunks
    answer = generate_answer(question, combined_context)
    return answer


In [ ]:
def retrieve_context(question, k=3):
    q_vec = embedder.encode([question]).astype('float32')
    distances, indices = index.search(q_vec, k)
    return [df.iloc[i]['long_answers'] for i in indices[0]]


In [ ]:
print(rag_answer("who is the president of un general assembly", k=5))
import random

# ✅ Step 10: Evaluate RAG vs Original T5 Baseline
sample_questions = [
    "who is the president of un general assembly",
    "what is the capital of ghana",
    "when was covid 19 first declared a pandemic",
    "who painted the mona lisa",
    "what is the largest planet in our solar system",
    "who wrote romeo and juliet",
    "where is mount everest located",
    "when did world war 2 end",
    "what language is spoken in brazil",
    "who founded microsoft"
]

# Ground truth answers (for reference)
true_answers = [
    "dennis francis",
    "accra",
    "march 11, 2020",
    "leonardo da vinci",
    "jupiter",
    "william shakespeare",
    "nepal",
    "1945",
    "portuguese",
    "bill gates"
]

# Evaluate both models
results = []
for i, q in enumerate(sample_questions):
    print(f"🧠 Q{i+1}: {q}")
    rag = rag_answer(q, k=5)
    
    print("💬 RAG Answer:", rag)
    print("✅ True:", true_answers[i])
    
    match = true_answers[i].lower() in rag.lower()
    print("🎯 Match:", "Yes" if match else "No")
    print("-" * 80)
    results.append(match)

# Compute accuracy
accuracy = (sum(results) / len(results)) * 100
print(f"\n📊 Overall RAG Accuracy: {accuracy:.2f}%")


In [ ]:
extra_context = "The current President of the UN General Assembly (2024–2025) is Dennis Francis of Trinidad and Tobago."
print(rag_answer("who is the president of un general assembly", k=5) + "\n" + extra_context)


In [ ]:
# Step 11 – Live Context Augmentation for RAG

# 1. Install and import Wikipedia API library
!pip install wikipedia-api

import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(user_agent='DekkyVoyages-RAG', language='en')

def fetch_wiki_context(topic, sentences=2):
    page = wiki_wiki.page(topic)
    if page.exists():
        # return summary (first N sentences) + fulltext
        summary = page.summary
        return summary
    else:
        return ''

# 2. Augmented retrieval function
def retrieve_augmented_context(question, k=3):
    # retrieve from your indexed FAISS dataset
    contexts = retrieve_context(question, k)
    # extract a keyword/topic from question (simple heuristic)
    # You might improve with proper keyword extraction
    topic = question
    wiki_context = fetch_wiki_context(topic, sentences=2)
    if wiki_context:
        contexts.insert(0, wiki_context)  # add wiki context first
    combined = " [SEP] ".join(contexts)
    return combined

# 3. Augmented RAG answer function
def rag_answer_augmented(question, k=3):
    combined_context = retrieve_augmented_context(question, k)
    answer = generate_answer(question, combined_context)
    return answer

# 4. Test with a current-events question
query = "who is the president of un general assembly"
print("🧠 Question:", query)
print("💬 Augmented RAG Answer:", rag_answer_augmented(query, k=5))


In [ ]:
def fetch_wiki_context(topic, sentences=2):
    import wikipedia
    try:
        summary = wikipedia.summary(topic, sentences=sentences, auto_suggest=True)
        return summary
    except Exception as e:
        return f"(No wiki data found: {e})"

def rag_answer_augmented(question, k=3):
    contexts = retrieve_context(question, k)
    wiki_context = fetch_wiki_context(question, sentences=3)
    combined_context = wiki_context + " [SEP] " + " [SEP] ".join(contexts)
    answer = generate_answer(question, combined_context)
    return answer

query = "who is the president of un general assembly"
print("🧠 Question:", query)
print("💬 Improved Augmented RAG Answer:", rag_answer_augmented(query, k=5))


In [ ]:
!pip install wikipedia


In [ ]:
def fetch_wiki_context(topic, sentences=2):
```)  

That’ll activate the Wikipedia-enhanced RAG pipeline.  
Let’s see if it now pulls **Dennis Francis (2024–2025)** as the correct UN General Assembly president! 🚀


In [ ]:
def fetch_wiki_context(topic, sentences=2):
    import wikipedia
    try:
        summary = wikipedia.summary(topic, sentences=sentences, auto_suggest=True)
        return summary
    except Exception as e:
        return f"(No wiki data found: {e})"

def rag_answer_augmented(question, k=3):
    contexts = retrieve_context(question, k)
    wiki_context = fetch_wiki_context(question, sentences=3)
    combined_context = wiki_context + " [SEP] " + " [SEP] ".join(contexts)
    answer = generate_answer(question, combined_context)
    return answer

query = "who is the president of un general assembly"
print("🧠 Question:", query)
print("💬 Improved Augmented RAG Answer:", rag_answer_augmented(query, k=5))


In [ ]:
wiki_context = fetch_wiki_context("President of the United Nations General Assembly", sentences=5)


In [ ]:
# ✅ Add the latest UN General Assembly President info
new_row = {
    "question": "who is the president of the un general assembly",
    "long_answers": "Annalena Baerbock of Germany was elected President of the United Nations General Assembly (80th session) on 2 June 2025 and will assume office on 9 September 2025."
}

# Append the new data
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Save updated dataset
updated_path = f"{path}/Natural-Questions-Filtered-Updated.csv"
df.to_csv(updated_path, index=False)

print(f"✅ New fact added and saved to: {updated_path}")


In [ ]:
# ✅ Rebuild FAISS Index after adding new data
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Reload the updated dataset
df = pd.read_csv(updated_path)

# Re-embed all long answers
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(df['long_answers'].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings).astype('float32')

# Create new FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"✅ Re-indexed {index.ntotal} contexts successfully.")


In [ ]:
  faiss.write_index(index, f"{path}/natural_questions_index.faiss")
print("✅ FAISS index saved successfully!")


In [ ]:
import gradio as gr

# ✅ Function that powers the RAG QA system
def answer_question(query):
    try:
        rag_result = rag_answer_augmented(query, k=5)
        return f"💬 **Question:** {query}\n\n🧠 **Answer:** {rag_result}"
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# ✅ Build the Gradio Interface
demo = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="markdown",
    title="🧠 RAG QA System with FLAN-T5 + Wikipedia",
    description="Ask factual questions and get context-aware answers powered by FAISS, FLAN-T5, and Wikipedia."
)

# ✅ Launch the app
demo.launch(debug=True)


In [ ]:
!pip install gradio


In [ ]:
def fetch_wiki_context(topic, sentences=2):
    import wikipedia
    try:
        summary = wikipedia.summary(topic, sentences=sentences, auto_suggest=True)
        return summary
    except Exception as e:
        return f"(No wiki data found: {e})"

def rag_answer_augmented(question, k=3):
    contexts = retrieve_context(question, k)
    wiki_context = fetch_wiki_context(question, sentences=3)
    combined_context = wiki_context + " [SEP] " + " [SEP] ".join(contexts)
    answer = generate_answer(question, combined_context)
    return answer


In [ ]:
import gradio as gr

# ✅ Function that powers the RAG QA system
def answer_question(query):
    try:
        rag_result = rag_answer_augmented(query, k=5)
        return f"💬 **Question:** {query}\n\n🧠 **Answer:** {rag_result}"
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# ✅ Build the Gradio Interface
demo = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="markdown",
    title="🧠 RAG QA System with FLAN-T5 + Wikipedia",
    description="Ask factual questions and get context-aware answers powered by FAISS, FLAN-T5, and Wikipedia."
)


In [ ]:
demo.launch(debug=True)


In [ ]:
pip install transformers torch sentence-transformers faiss-cpu wikipedia gradio


In [ ]:
!pip install transformers faiss-cpu gradio sentence-transformers kagglehub torch


In [1]:
import sys
print(sys.executable)


C:\Users\udeko\anaconda3\python.exe


In [4]:
!pip install transformers faiss-cpu gradio sentence-transformers kagglehub torch


In [5]:
import kagglehub

# Download latest Natural Questions dataset
path = kagglehub.dataset_download("frankossai/natural-questions-dataset")
print("✅ Dataset downloaded to:", path)


✅ Dataset downloaded to: C:\Users\udeko\.cache\kagglehub\datasets\frankossai\natural-questions-dataset\versions\1


In [6]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import faiss, torch

# Load models
embedder = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

print("✅ Embedding model and FLAN-T5 loaded successfully!")


✅ Embedding model and FLAN-T5 loaded successfully!


In [7]:
import numpy as np

# Example docs (you can expand this later)
docs = [
    "Penicillin was discovered by Alexander Fleming in 1928.",
    "The Apollo 11 mission landed on the moon in 1969.",
    "Nigeria’s current president is Bola Ahmed Tinubu."
]

# Create embeddings
doc_embeddings = embedder.encode(docs)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

def retrieve_context(query):
    q_emb = embedder.encode([query])
    _, idx = index.search(np.array(q_emb), 1)
    return docs[idx[0][0]]


In [8]:
def generate_answer(query):
    context = retrieve_context(query)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
import gradio as gr

def rag_pipeline(query):
    return generate_answer(query)

demo = gr.Interface(
    fn=rag_pipeline,
    inputs=gr.Textbox(label="Ask a Question"),
    outputs="text",
    title="🧠 The RAG-AI-Assistant",
    description="Ask factual questions — powered by FAISS, FLAN-T5, and Sentence Transformers."
)

demo.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
